In [1]:
import pandas as pd
import numpy as np
from collections import deque
import random

In [2]:
file_path = '../bar_movement/data/'
currency_pair = 'Eur_Usd'

In [3]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2015-2023.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

In [5]:
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000

prices = df[['Date', 'Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume']]
df.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', 'Volume'], axis=1, inplace=True)

assert len(prices) == len(df)

In [6]:
cutoff_date = '2022-06-15 05:00:00'

df_train = df.loc[df['Date'] <= cutoff_date]
df_train.reset_index(drop=True, inplace=True)
prices_train = prices.loc[prices['Date'] <= cutoff_date]
prices_train.reset_index(drop=True, inplace=True)

df_test = df.loc[df['Date'] > cutoff_date]
df_test.reset_index(drop=True, inplace=True)
prices_test = prices.loc[prices['Date'] > cutoff_date]
prices_test.reset_index(drop=True, inplace=True)

df_train.drop(['Date'], axis=1, inplace=True)

/Users/mymac/miniforge3/envs/forex/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
validation_avg_error = (abs(prices_train['Mid_High'] - prices_train['Mid_Low']).mean() / 2) * pips_multiplier
validation_avg_error

15.073042002934708

In [13]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]
rounding = 3 if 'Jpy' in currency_pair else 5
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2022-2023.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(pips_to_risk, adapt_errors, error_array_len, error_multiplier, invert):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    prev_long_date, trade = None, None
    sl_pips = pips_to_risk / pips_multiplier

    ask_pips_up_errors, ask_pips_down_errors, bid_pips_up_errors, bid_pips_down_errors = deque(maxlen=error_array_len), deque(maxlen=error_array_len), deque(maxlen=error_array_len), deque(maxlen=error_array_len)

    for _ in range(error_array_len):
        ask_pips_up_errors.append(validation_avg_error)
        ask_pips_down_errors.append(validation_avg_error)
        bid_pips_up_errors.append(validation_avg_error)
        bid_pips_down_errors.append(validation_avg_error)

    for i in range(len(df)):
        curr_date, curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Date', 'Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        curr_long_df = df_test.loc[df_test.Date <= curr_date]
        gte = df_test.loc[df_test.Date >= curr_date]

        if len(gte) == 0:
            break

        curr_long_df_len = len(curr_long_df)

        if curr_long_df_len < 2:
            continue

        curr_long_date = curr_long_df.loc[curr_long_df.index[-2], 'Date']

        if trade is None and curr_long_date != prev_long_date:
            prev_long_date = curr_long_date

            ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred = validation_avg_error, validation_avg_error, validation_avg_error, validation_avg_error

            ask_pips_up_error_avg = np.array(ask_pips_up_errors).mean() if adapt_errors else validation_avg_error
            ask_pips_down_error_avg = np.array(ask_pips_down_errors).mean() if adapt_errors else validation_avg_error
            bid_pips_up_error_avg = np.array(bid_pips_up_errors).mean() if adapt_errors else validation_avg_error
            bid_pips_down_error_avg = np.array(bid_pips_down_errors).mean() if adapt_errors else validation_avg_error

            if adapt_errors:
                curr_prices_long = prices_test.loc[prices_test.Date <= curr_date]
                ask_open, ask_high, ask_low, bid_open, bid_high, bid_low = curr_prices_long.loc[curr_prices_long.index[-1], ['Ask_Open', 'Ask_High', 'Ask_Low', 'Bid_Open', 'Bid_High', 'Bid_Low']]
                ask_pips_up_true, ask_pips_down_true, bid_pips_up_true, bid_pips_down_true = abs(ask_high - ask_open) * pips_multiplier, abs(ask_open - ask_low) * pips_multiplier, abs(bid_high - bid_open) * pips_multiplier, abs(bid_open - bid_low) * pips_multiplier

                ask_pips_up_errors.append(abs(ask_pips_up_true - ask_pips_up_pred))
                ask_pips_down_errors.append(abs(ask_pips_down_true - ask_pips_down_pred))
                bid_pips_up_errors.append(abs(bid_pips_up_true - bid_pips_up_pred))
                bid_pips_down_errors.append(abs(bid_pips_down_true - bid_pips_down_pred))

            
            ask_pips_up_pred = ask_pips_up_pred + (ask_pips_up_error_avg * error_multiplier)
            ask_pips_down_pred = ask_pips_down_pred + (ask_pips_down_error_avg * error_multiplier)
            bid_pips_up_pred = bid_pips_up_pred + (bid_pips_up_error_avg * error_multiplier)
            bid_pips_down_pred = bid_pips_down_pred + (bid_pips_down_error_avg * error_multiplier)

            buy_signal = max([ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred]) == bid_pips_up_pred and bid_pips_up_pred >= pips_to_risk and bid_pips_down_pred < pips_to_risk
            sell_signal = max([ask_pips_up_pred, ask_pips_down_pred, bid_pips_up_pred, bid_pips_down_pred]) == ask_pips_down_pred and ask_pips_down_pred >= pips_to_risk and ask_pips_up_pred < pips_to_risk

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal

            if buy_signal:
                open_price = float(curr_ao)
                stop_loss = round(open_price - sl_pips, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1

            elif sell_signal:
                open_price = float(curr_bo)
                stop_loss = round(open_price + sl_pips, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [15]:
pips_to_risk_vals = [20, 30, 40, 50] 
error_multipliers = [0.0, 0.5, 1.0, 1.5, 2.0]
adapt_errors_vals = [True, False]
error_array_lens = [5, 10, 20] 
invert_vals = [True, False]

all_combos = []

for pips_to_risk in pips_to_risk_vals:
    for error_multiplier in error_multipliers:
        for invert in invert_vals:
            for adapt_errors in adapt_errors_vals:
                for err_array_len in error_array_lens:
                    error_array_len = err_array_len if adapt_errors else 1
                    all_combos.append((pips_to_risk, adapt_errors, error_array_len, error_multiplier, invert))

                    if not adapt_errors:
                        break

best_pips_to_risk, best_adapt_errors, best_error_array_len, best_error_multiplier, best_invert_val = None, None, None, None, None
top_n_results, best_rewards, best_reward, runs_finished = 10, [], -np.inf, 0

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for pips_to_risk, adapt_errors, error_array_len, error_multiplier, invert in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(pips_to_risk, adapt_errors, error_array_len, error_multiplier, invert)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'pips_to_risk': pips_to_risk, 'adapt_errors': adapt_errors, 'error_array_len': error_array_len, 'error_multiplier': error_multiplier, 'invert': invert})

    if total_profit > best_reward:
        best_reward = total_profit
        best_pips_to_risk, best_adapt_errors, best_error_array_len, best_error_multiplier, best_invert_val = pips_to_risk, adapt_errors, error_array_len, error_multiplier, invert
 
    print('Best reward so far: ' + str(best_reward))
    print()


Num runs: 160

-2599.896000000006 -64.33076000000001 -2664.226760000006
Num buys: 69
Num sells: 71
Num trades: 140
Num wins: 22
Num losses: 94
Win streak: 4
Loss streak: 26
Avg pips risked: 0.0029999999999999957
Avg win amount: 95.45072727272645
Min win amount: 49.99799999999819
Max win amount: 249.98999999999836
Avg loss amount: -39.82891525423718
Min loss amount: -49.998000000001895
Max loss amount: 0.0
Remaining runs: 159
Best reward so far: -2664.226760000006

0 0 0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 158
Best reward so far: 0

-99.99599999999859 -17.4987 -117.49469999999859
Num buys: 22
Num sells: 7
Num trades: 29
Num wins: 5
Num losses: 18
Win streak: 4
Loss streak: 11
Avg pips risked: 0.004
Avg win amount: 159.98720000000014
Min win amount: 99.99200000000009
Max win amount: 249.98000000000022
Avg loss amount: -37.497166666666644
Min loss amount: -49.99999999999866
Max loss amount: 0.0
Remaining runs: 157
B

In [16]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best adapt errors val: ' + str(best_adapt_errors))
print('Best error array length: ' + str(best_error_array_len))
print('Best best error multiplier: ' + str(best_error_multiplier))
print('Best best invert val: ' + str(best_invert_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1449.4400200000016
Best pips to risk: 20
Best adapt errors val: True
Best error array length: 10
Best best error multiplier: 0.5
Best best invert val: True
-----------------------
Top results:
{'reward': 1195, 'pips_to_risk': 40, 'adapt_errors': True, 'error_array_len': 5, 'error_multiplier': 1.5, 'invert': True}
{'reward': 613, 'pips_to_risk': 50, 'adapt_errors': True, 'error_array_len': 10, 'error_multiplier': 2.0, 'invert': False}
{'reward': 639, 'pips_to_risk': 50, 'adapt_errors': True, 'error_array_len': 5, 'error_multiplier': 2.0, 'invert': True}
{'reward': 810, 'pips_to_risk': 50, 'adapt_errors': True, 'error_array_len': 10, 'error_multiplier': 1.5, 'invert': False}
{'reward': 686, 'pips_to_risk': 30, 'adapt_errors': True, 'error_array_len': 10, 'error_multiplier': 1.0, 'invert': True}
{'reward': 688, 'pips_to_risk': 50, 'adapt_errors': True, 'error_array_len': 5, 'error_multiplier': 1.5, 'invert': True}
{'reward': 1348, 'pips